In [1]:
# ! curl -L -o gtsrb.zip https://cdn.cs50.net/ai/2023/x/projects/5/gtsrb.zip
# ! unzip gtsrb.zip

In [1]:
# Import Dependencies 
import os 
import cv2 
import numpy as np 
from tqdm import tqdm 

from typing import List, Tuple, Generator, Any 

import torch 
import torch.nn as nn
from torch.utils.data import IterableDataset, DataLoader 

In [2]:
# Constants 
IMG_WIDTH = 30 
IMG_HEIGHT = 30 
TEST_SIZE = 0.3 
NUM_CATEGORIES = 43

In [37]:
def load_data(train: bool = False) -> Generator[Any, Any, Any]: 
    """ 
    Helper function 

    Loads images from directory 
    """ 
    data_dir = "gtsrb"
    directories = os.listdir(data_dir)   
    for directory in directories: 
        img_folder = os.path.join(data_dir, directory) 
        image_files = os.listdir(img_folder)  
        train_size = int((1 - TEST_SIZE) * (len(image_files))) 
        print(train_size)
        if train: 
            files = image_files[:train_size]
        else:
            files = image_files[train_size:]  

        for image in files: 
            image_path = os.path.join(img_folder, image)
            image_contents = cv2.imread(image_path) 
            image_contents = cv2.resize(image_contents, (IMG_WIDTH, IMG_HEIGHT)) 
            image_contents = np.array(image_contents, dtype='float')  
            image_contents = np.transpose(image_contents, (2, 0, 1)) 
            # Normalize pixel values if needed (optional)
            image_contents /= 255.0  # Normalize pixel values to range [0, 1]

            # Convert to PyTorch tensor
            image_tensor = torch.tensor(image_contents, dtype=torch.float32)
            yield (image_tensor, directory)

In [38]:
class Dataset(IterableDataset):
    def __init__(self, generator): 
        self.generator = generator 
    
    def __iter__(self): 
        return iter(self.generator) 

dataset = Dataset(load_data()) 

In [39]:
train_loader = DataLoader(dataset=Dataset(load_data(train=True)), batch_size=10)  
test_loader = DataLoader(dataset=Dataset(load_data(train=False)), batch_size=100)

In [40]:
class CNN(nn.Module): 
    def __init__(self, out_1=16, out_2=32):
        super(CNN, self).__init__() 
        # input (30, 30, 3)
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=out_1, kernel_size=5, padding=2)  # Output: (30, 30, out_1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)  # Output (15, 15, out_1)

        self.cnn2 = nn.Conv2d(in_channels=out_1, out_channels=out_2, kernel_size=5, stride=1, padding=2)  # Output(15, 15, out_2)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)  # Output(7, 7, out_2)

        self.fc1 = nn.Linear(out_2 * 7 * 7, NUM_CATEGORIES)

    def forward(self, x): 
        x = self.cnn1(x) 
        x = torch.relu(x)
        x = self.maxpool1(x)
        x = self.cnn2(x)
        x = torch.relu(x)
        x = self.maxpool2(x)
        x = self.view(x.size(0), -1)
        x = self.fc1(x)
        return x 
    
    def activations(self, x): 
        z1 = self.cnn1(x)
        a1 = torch.relu(z1) 
        out = self.maxpool1(a1) 

        z2 = self.cnn2(out)
        a2 = torch.relu(z2)
        out1 = self.maxpool2(a2) 
        out = out.view(out.size(0), -1)
        return z1, a1, z2, a2, out1, out
    
model = CNN(out_1=16, out_2=32)

In [41]:
# Criterion 
criterion = nn.CrossEntropyLoss() 
learning_rate = 0.1 
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [56]:
import matplotlib.pyplot as plt
import numpy as np

def show_data(data):
    image, label = data
    print(image.shape)
    print(image.dtype)
    
    # Convert from tensor to numpy
    image = image.numpy()
    
    # Reshape the image from (C, H, W) to (H, W, C)
    image = np.transpose(image, (1, 2, 0))
    
    # If the image was normalized, denormalize it (optional)
    if image.max() <= 1.0:
        image = image * 255.0  # Rescale to original range
    
    # Convert to uint8 if necessary
    image = image.astype(np.uint8)
    
    plt.imshow(image)
    plt.title(label)
    plt.axis('off')  # Turn off axis
    plt.show()

# Example usage
# Assuming `data` is a tuple (image_tensor, label)
# show_data((image_tensor, 'Example Label'))
